In [1]:
import pickle

models = ["LLama", "Mistral", "Aya"]
languages = ["English", "Dutch", "Chinese", "Italian"]

In [7]:
templates_list = []
for language in languages:
    with open(f"../../templates/templates_{language}.pkl", "rb") as f:
        templates = pickle.load(f)
        for template in templates:
            templates_list.append(template[0]["content"])

In [9]:
model_output_list = {"LLama": [], "Mistral": [], "Aya": []}

pre_text = "Is the following text specific or general? The text is: "
for model in models:
    results = []
    for language in languages:
        with open(f"../../out/results_{model}_{language}.pkl", "rb") as f:
            outputs = pickle.load(f)

        for item in outputs:
            model_output_list[model].append(item)
            results.append([{"role": "user", "content": pre_text + item}])

In [2]:
model_classification_list = {"LLama": [], "Mistral": [], "Aya": []}
for model in models:
    with open(f"results_general_vs_specific_{model}.pkl", "rb") as f:
        classifications = pickle.load(f)
        for classification in classifications:
            model_classification_list[model].append(classification)

In [3]:
classified = {"LLama": [], "Mistral": [], "Aya": []}
topics = ["general", "specific"]
specific_texts = [
    "The text is ",
    "This text is ",
    "this text as "
]

In [4]:
for model in models:
    parsed_counts = 0
    for item in model_classification_list[model]:
        before_count = len(classified[model])
        topic_count = 0
        text_count = -1
        for topic in topics:
            if topic in item:
                topic_count += 1
        if topic_count == 1:
            for topic in topics:
                if topic in item:
                    classified[model].append(topic)

        if topic_count > 1 or topic_count == 0:
            if topic_count > 1:
                text_count = 0
                for text in specific_texts:
                    if text in item:
                        text_after = item.split(text)[1][:9].replace(".", "").replace(" ", "")
                        for topic in topics:
                            if topic in text_after:
                                classified[model].append(topic)
                                text_count += 1
                        if text_after == "amixof":
                            classified[model].append("Both / None" )
                            text_count += 1
                        if "Il testo è specifico" == item[:20]:
                            print(1)
                            classified[model].append("specific")
                            text_count += 1
                if text_count == 0:
                    if "Il testo è specifico" == item[:20]:
                        classified[model].append("specific")
                        text_count += 1

                    if "generalista" in item:
                        classified[model].append("general")
                        text_count += 1

                if text_count == 0:
                    print(item)

In [5]:
set(classified["LLama"])

{'Both / None', 'general', 'specific'}

In [10]:
for model in models:
    with open(f"classification_{model}.csv", "w") as f:
        f.write("id,model_id,template_id,template,output,classification\n")

        for i in range(len(templates_list)):
            model_output = (
                model_output_list[model][i].replace("\n", "\\n").replace('"', "'")
            )
            f.write(
                f'{i},"{model}",{i},"{templates_list[i]}","{ model_output}","{classified[model][i]}"\n'
            )
